In [1]:
import torch
import torchvision
from torch import nn
import json
from utils import *
import model
import dataset

import imageio
import scipy
from imageio import imread
import os
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pack_padded_sequence
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import torch
from imageio import imread
from PIL import Image
import PIL
import pickle

In [2]:
def set_seed(seed: int = 666) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.set_deterministic(True)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()

In [3]:
with open('images/WORDMAP_COCO.json') as f:
    wordmap = json.load(f)

In [4]:
checkpoint = torch.load('checkpoint_17_3000.pth.tar')
start_epoch = checkpoint['epoch']
dec = checkpoint['decoder']
optimizer_decoder = checkpoint['decoder_optimizer']
enc = checkpoint['encoder']
optimizer_encoder = checkpoint['encoder_optimizer']

In [53]:
img = imread("ggg.png")
if len(img.shape) == 2:
    img = img[:, :, np.newaxis]
    img = np.concatenate([img, img, img], axis=2)
img = Image.fromarray(img)
img = np.array(img.resize((256, 256), PIL.Image.BICUBIC))
img = img.transpose(2, 0, 1)
assert img.shape == (3, 256, 256)
assert np.max(img) <= 255

In [54]:
img.shape

(3, 256, 256)

In [55]:
img = torch.FloatTensor(img/255.).unsqueeze(0)

In [56]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [57]:
transform = transforms.Compose([normalize])

In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img = transform(img).to(device)

In [59]:
res = dict((v,k) for k,v in wordmap.items())

In [60]:
with torch.no_grad():
    enc = enc.eval()
    dec = dec.eval()
    predictions = []
    encoder_out = enc(img)
    encoder_out = encoder_out.view(1, -1, 2048)
    h = dec.h_init(encoder_out.mean(dim = 1)) # (batch_size, decoder_hidden_size)
    c = dec.c_init(encoder_out.mean(dim = 1)) # (batch_size, decoder_hidden_size)
    last_word = torch.tensor(wordmap['<start>']).unsqueeze(0).unsqueeze(0).to(device)
    while last_word != wordmap['<end>']:

        print(res[int(last_word)])
        # Выбираем эмбеддинг слова, стоящего на позиции last_word
        decoder_out = dec.embedding(last_word)[:, 0]

        # Механизм внимания
        encoder_weighted = dec.Attention(batch_size = 1,
                                          encoder_out = encoder_out,
                                          decoder_out = decoder_out)

        # Конкатенируем информцию из механизма внимания и информацию о предыдущем слове
        decoder_in = torch.cat((encoder_weighted, decoder_out), 1)

        # Предсказываем вероятности появления слов на текущей позиции
        h, c = dec.LSTMCell(decoder_in, (h, c)) # (batch_size, decoder_hidden_size)
        predictions_word = dec.linear(h) # (batch_size, decoder_hidden_size)
        predictions.append(predictions_word)
        maxword = torch.argmax(predictions_word)
        last_word = maxword.unsqueeze(0).unsqueeze(0).to(device)

<start>
a
large
amount
of
various
types
of
vegetables


In [ ]:
tt = open('glove.6B.50d.txt', 'r', encoding='utf-8').read()
tt = tt.split('\n')

In [ ]:
words = [pp.split(' ')[0] for pp in tt]
encodings = [pp.split(' ')[1:] for pp in tt]

In [ ]:
new_wordmap = {word: enc for word, enc in zip(words, encodings)}

In [ ]:
encodings = []
for word in wordmap:
    try:
        encodings.append(new_wordmap[word])
    except:
        print(word)

In [ ]:
words

In [ ]:
encodings

Для новых слов тупо зарандобмить